## !!! Deprecated
## I cannot find a way to downgrade python version to 3.7 on Colab

### Change python version to 3.7

[Op]: Select python3.7 version

In [ ]:
!sudo apt-get install python3.7
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3

In [ ]:
!python --version

In [ ]:
!sudo apt install python3-pip
!python -m pip install --upgrade pip --user
!sudo apt-get install python3.7-distutils

In [ ]:
!git clone https://github.com/XinGuu/CTAB-GAN.git

In [ ]:
import os
os.chdir('CTAB-GAN')
!pip install -r requirements.txt

In [ ]:
# Importing the model
from model.ctabgan import CTABGAN
# Importing the evaluation metrics 
from model.eval.evaluation import get_utility_metrics,stat_sim,privacy_metrics
# Importing standard libraries
import numpy as np
import pandas as pd
import glob
import argparse

In [ ]:
parser = argparse.ArgumentParser()
# Required parameters
parser.add_argument("--dataset", type=str, choices=["adult", "bank"], default="adult")
parser.add_argument("--num_exp", type=int, default=1, help="Specifying the replication number")

args = parser.parse_args()
# Specifying the replication number
num_exp = args.num_exp
# Specifying the name of the dataset used
dataset = args.dataset

root_dir = '../'
# Specifying the path of the dataset used
real_path = f"{root_dir}/datasets/raw/{dataset}/train.csv"
# Specifying the root directory for storing generated data
fake_file_root = "Fake_Datasets"

In [ ]:
# load attributs
import json

with open(f"datasets/raw/{dataset}/attrs.json", 'r') as f:
    attrs = json.load(f)

categorical_columns = attrs["categorical"]
integer_columns = attrs["numerical"]
problem_type = {"Classification": attrs["target"]}

In [ ]:
# Initializing the synthesizer object and specifying input parameters
# Notice: If you have continuous variable, you do not need to explicitly assign it. It will be treated like 
# that by default
synthesizer =  CTABGAN(raw_csv_path = real_path,
                 test_ratio = 0.20,  
                 categorical_columns = categorical_columns,
                 log_columns = [],
                 mixed_columns= {},
                 integer_columns = integer_columns,
                 problem_type= problem_type,
                 epochs = 150) 

# Fitting the synthesizer to the training dataset and generating synthetic data
for i in range(num_exp):
    synthesizer.fit()
    syn = synthesizer.generate_samples()
    syn.to_csv(fake_file_root+"/"+dataset+"/"+ dataset+"_fake_{exp}.csv".format(exp=i), index= False)

In [ ]:
# Collecting the paths to all corresponding generated datasets for evaluation 
fake_paths = glob.glob(fake_file_root+"/"+dataset+"/"+"*")

#### ML Utility Evaluation

In [ ]:
# Specifying the list of classifiers to conduct ML utility evaluation
classifiers_list = ["lr","dt","rf","mlp","svm"]

# Storing and presenting the results as a dataframe
result_mat = get_utility_metrics(real_path,fake_paths,"MinMax",classifiers_list, test_ratio = 0.20)
result_df  = pd.DataFrame(result_mat,columns=["Acc","AUC","F1_Score"])
result_df.index = classifiers_list
result_df

,Acc,AUC,F1_Score
lr,1.064592,0.009517,0.061383
dt,6.285188,0.063739,0.071529
rf,2.589825,0.027153,0.040049
mlp,2.763845,0.013811,0.126618
svm,2.896919,0.049234,0.124052


#### Statistical Similarity Evaluation

In [ ]:
# Specifying the categorical columns of the dataset used
adult_categorical = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']

# Storing and presenting the results as a dataframe
stat_res_avg = []
for fake_path in fake_paths:
    stat_res = stat_sim(real_path,fake_path,adult_categorical)
    stat_res_avg.append(stat_res)

stat_columns = ["Average WD (Continuous Columns","Average JSD (Categorical Columns)","Correlation Distance"]
stat_results = pd.DataFrame(np.array(stat_res_avg).mean(axis=0).reshape(1,3),columns=stat_columns)
stat_results

,Average WD (Continuous Columns,Average JSD (Categorical Columns),Correlation Distance
0,0.009362,0.1204,0.761534


#### Nearest Neighbour Privacy Analysis

In [ ]:
# Storing and presenting the results as a dataframe
priv_res_avg = []
for fake_path in fake_paths:
    priv_res = privacy_metrics(real_path,fake_path)
    priv_res_avg.append(priv_res)
    
privacy_columns = ["DCR between Real and Fake (5th perc)","DCR within Real(5th perc)","DCR within Fake (5th perc)","NNDR between Real and Fake (5th perc)","NNDR within Real (5th perc)","NNDR within Fake (5th perc)"]
privacy_results = pd.DataFrame(np.array(priv_res_avg).mean(axis=0).reshape(1,6),columns=privacy_columns)
privacy_results

,DCR between Real and Fake (5th perc),DCR within Real(5th perc),DCR within Fake (5th perc),NNDR between Real and Fake (5th perc),NNDR within Real (5th perc),NNDR within Fake (5th perc)
0,0.485676,0.216545,0.22867,0.632722,0.442052,0.431408


In [ ]:
# Storing generated data for future use if needed
syn.to_csv(fake_file_root+"/"+dataset+"/"+ dataset+"_fake_{exp}.csv".format(exp=i), index= False)